In [ ]:
import numpy as np

import sys
from pathlib import Path

import torch

from sklearn.model_selection import train_test_split
import wandb

In [ ]:
# Importerer spaleron og micro svarte hull data

module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
import trainers
import nnmodels

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [ ]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [ ]:
print(bhArray.shape)
print(sphArray.shape)

In [ ]:
# Vi slår sammen dataene for å ha et samlet datasett som kjører gjennom modellen
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [ ]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
# 75% av datasettet havner i trainData, 25% havner i testData, 
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [ ]:
# gjør det om til en tensor fra numpy array. Vi gjør dette for at at dataene skal være lagret på GPU en istedet for en liste som er lagret på CPUen 
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [ ]:
# to tensor datasett 
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [ ]:
from torchvision import models
from torchsummary import summary

model = nnmodels.ConvModel(0).to(device)
summary(model, (3, 50, 50))

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'model': {'values': ['cnn']},
        'loss': {'values': ['cross', 'customLoss-3', 'customLoss-4', 'customLoss-5', 'customLoss-6', 'customLoss-7', 'customLoss-8', 'customLoss-9']},
        'epoch': {'values': [20]},
        'batch_size': {'values': [50]},
        'gamma': {'values': [1]},
        'learning_rate': {'values': [0.01]},
        'optimizer': {'values': ['adam']},
        'dropout': {'values': [0]},
     }
}

In [ ]:
#wrapper function
def trainFunction():
    with wandb.init(project="Sweep", name="Sweep") as run:
        config = wandb.config

        name_str = config['model'] + "_" + config['loss'] + "_" + config['optimizer']
        run.name = name_str
        
        trainers.sweep(train, test, config)

In [14]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="Sweep")
wandb.agent(sweep_id, function=trainFunction)